In [1]:
from __future__ import absolute_import
from __future__ import division

In [6]:
SEED=1234
import numpy as np
import math
from matplotlib import pyplot as plt
np.random.seed(SEED)
import keras
from keras import backend as K
import tensorflow as tf
import os, shutil, scipy.io
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Activation, add, Dropout, Lambda, Input, average, LSTM
from tensorflow.keras.layers import BatchNormalization
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras import optimizers
from keras import regularizers


In [8]:
from os import makedirs
from numpy import dstack, mean, std, argmax, tensordot, array
from numpy.linalg import norm
from tensorflow.keras.layers import concatenate
from keras.utils import plot_model
from scipy.optimize import differential_evolution
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
import timeit

In [11]:
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Activation, add, Dropout, Lambda, LSTM, Flatten
from keras.layers import Input, average, TimeDistributed, SimpleRNN, LeakyReLU, GlobalAveragePooling1D
from keras.layers.convolutional import Conv1D, MaxPooling1D, AveragePooling1D
from keras import backend as K

In [12]:
caseNo = 118
weight_4_mag = 100
weight_4_ang = 180/math.pi

psse_data = scipy.io.loadmat('data_for_SE_case'+str(caseNo)+'_for_ML.mat')

In [19]:
data_x = psse_data['Z_measure']
data_y = psse_data['Output']
print(data_x.shape)
data_x=data_x[0:data_y.shape[0], :] # this is because measurement are available for 4 and half years but state estimation is available only for two years for IEEE 118
print(data_x.shape)

(39444, 562)
(17520, 562)


In [31]:
data_y[:, 0:caseNo] = weight_4_mag*data_y[:, 0:caseNo]
data_y[:, caseNo:] = weight_4_ang*data_y[:, caseNo:]

In [32]:
split_train = int(0.6*data_x.shape[0])
test_x=data_x[:split_train, :]
test_y=data_y[:split_train, :]
train_x=data_x[split_train:, :]
train_y=data_y[split_train:, :]

In [36]:
# calculate the hubber loss
def huber_loss(y_true, y_pred, clip_delta=1.0):
    error = y_true - y_pred
    cond  = K.abs(error) < clip_delta
    squared_loss = 0.5 * K.square(error)
    linear_loss  = clip_delta * (K.abs(error) - 0.5 * clip_delta)
    return tf.where(cond, squared_loss, linear_loss)
# calculate the hubber loss mean for the compiler
def huber_loss_mean(y_true, y_pred):
    return K.mean(huber_loss(y_true, y_pred))

In [37]:
input_shape = (train_x.shape[1],)
num_classes=train_y.shape[1]

In [42]:
def nn1_psse(input_shape, num_classes, weights=None):
    '''
    :param input_shape:
    :param num_classes:
    :param weights: 6 layers
    :return: 1 hidden layer NN model with specified training loss
    '''
    data = Input(shape=input_shape, dtype='float', name='data')
    dense1 = Dense(units = input_shape[0], activation='relu',  use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros')(data)
    dense2 = Dense(units = input_shape[0], activation='relu',  use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros')(dense1)
    dense3 = Dense(units = input_shape[0], activation='relu',  use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros')(dense2)
    dense4 = Dense(units = input_shape[0], activation='relu',  use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros')(dense3)
    dense5 = Dense(units = input_shape[0], activation='relu',  use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros')(dense4)

    dense6 = Dense(units = input_shape[0], activation='relu',  use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros')(dense5)
#    dense7 = Dense(units = input_shape[0], activation='relu',  use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros')(dense6)
#    dense8 = Dense(units = input_shape[0], activation='relu',  use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros')(dense7)
#    dense9 = Dense(units = input_shape[0], activation='relu',  use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros')(dense8)
#    dense10 = Dense(units = input_shape[0], activation='relu',  use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros')(dense9)

#    drop1 = Dropout(rate=0.5, name='drop1')(dense8)
    predictions = Dense(units = num_classes, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros')(dense6)

    model = Model(inputs=data, outputs=predictions)
    if weights is not None:
        model.load_weights(weights)
    #sgd = optimizers.adam(lr=0.001)

    sgd = optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)

    model.compile(optimizer=sgd, loss=huber_loss_mean,
                  metrics=['mae'])

    return model

In [47]:
epoch_num = 200
# # create directory for models
#makedirs('States_MLP_118') # For CNN


def fit_model_DNN(train_x, train_y, input_shape, num_classes):
    psse_model = nn1_psse(input_shape, num_classes, weights=None) # For DNN
    psse_model.fit(train_x, train_y, epochs=200, batch_size=64, verbose=2)
    return psse_model


# # fit and save model models to use later for ensembling
n_members=6
for i in range(n_members):
#     # fit model
     model = fit_model_DNN(train_x, train_y, input_shape, num_classes)
#    # save model
     filename = 'States_MLP_118/model_' + str(i + 1) + '.h5'  # for DNN
     model.save(filename)
     print('> Saved model %s' % filename)


Epoch 1/10
110/110 - 5s - loss: 6209.5156 - mae: 6210.0137 - 5s/epoch - 43ms/step
Epoch 2/10
110/110 - 3s - loss: 6156.8247 - mae: 6157.3232 - 3s/epoch - 26ms/step
Epoch 3/10
110/110 - 3s - loss: 6156.8169 - mae: 6157.3145 - 3s/epoch - 26ms/step
Epoch 4/10
110/110 - 3s - loss: 6156.8110 - mae: 6157.3096 - 3s/epoch - 26ms/step
Epoch 5/10
110/110 - 3s - loss: 6156.8027 - mae: 6157.3003 - 3s/epoch - 26ms/step
Epoch 6/10
110/110 - 3s - loss: 6156.7944 - mae: 6157.2930 - 3s/epoch - 26ms/step
Epoch 7/10
110/110 - 3s - loss: 6156.7832 - mae: 6157.2812 - 3s/epoch - 26ms/step
Epoch 8/10
110/110 - 3s - loss: 6156.7715 - mae: 6157.2695 - 3s/epoch - 27ms/step
Epoch 9/10
110/110 - 3s - loss: 6156.7534 - mae: 6157.2510 - 3s/epoch - 29ms/step
Epoch 10/10
110/110 - 3s - loss: 6156.6860 - mae: 6157.1836 - 3s/epoch - 26ms/step
> Saved model States_MLP_118/model_1.h5
Epoch 1/10
110/110 - 4s - loss: 6140.1094 - mae: 6140.6084 - 4s/epoch - 34ms/step
Epoch 2/10
110/110 - 3s - loss: 6156.8276 - mae: 6157.325

In [48]:
def load_all_models(n_models):
    all_models=list()
    for i in range(n_models):
        # define filename for this ensemble
        filename = 'States_MLP_'+str(caseNo)+'/model_' + str(i + 1) + '.h5'  # for Combined
        # load model from file
        #model=load_model(filename) # use this for MLP
        model=load_model(filename, custom_objects={'huber_loss_mean':huber_loss_mean})
        # add to list of members
        all_models.append(model)
        print('>loaded %s' % filename)
    return all_models

In [49]:
n_members=6
members=load_all_models(n_members)
print('Loaded %d models' %len(members))

>loaded States_MLP_118/model_1.h5
>loaded States_MLP_118/model_2.h5
>loaded States_MLP_118/model_3.h5
>loaded States_MLP_118/model_4.h5
>loaded States_MLP_118/model_5.h5
>loaded States_MLP_118/model_6.h5
Loaded 6 models


In [ ]:
split_train = int(0.6*test_x.shape[0])
mtrain_x=test_x[:split_train, :]
mtrain_y=test_y[:split_train, :]
test_x=test_x[split_train:, :]
test_y=test_y[split_train:, :]

#data_new=scipy.io.loadmat('States_onli_SE_case'+str(caseNo)+'_for_ML.mat')
data_new=scipy.io.loadmat('States_onli_SE_case'+str(caseNo)+'_for_ML_Non_Gaussian.mat')
data_testy=data_new['Output']
data_testx=data_new['Z_measure']

data_testy[:, 0:caseNo] = weight_4_mag*data_testy[:, 0:caseNo]
data_testy[:, caseNo:] = weight_4_ang*data_testy[:, caseNo:]

test_x=data_testx[0:data_testy.shape[0], :]
test_y=data_testy

In [51]:
vrmse=100000
for model in members:
    start = timeit.default_timer()
    yhat = model.predict(test_x)
    stop = timeit.default_timer()
    # print('Time: %f' % (stop - start))
    print('Per iteration time: %f' % ((stop - start) / test_x.shape[0] * 1000))


    RMSEv = np.sqrt(mean_squared_error(yhat[:, 0:caseNo], test_y[:, 0:caseNo]))
    print('Model Test data RMSEv: %.4f' % RMSEv)
    MAEv = mean_absolute_error(yhat[:, 0:caseNo], test_y[:, 0:caseNo])
    print('Model Test data MAEv: %.4f' % MAEv)

    RMSEa = np.sqrt(mean_squared_error(yhat[:, caseNo:], test_y[:, caseNo:]))
    print('Model Test data RMSEa: %.4f' % RMSEa)
    MAEa = mean_absolute_error(yhat[:, caseNo:], test_y[:, caseNo:])
    print('Model Test data MAEa: %.4f' % MAEa)

    if vrmse > RMSEv:
        yhat_save = yhat
        vrmse = RMSEv
        vmae = MAEv
        armse = RMSEa
        amae = MAEa
        

print('RMSE to win min model for voltage is : %.4f' % vrmse)
print('MAE to win min model for voltage is : %.4f' % vmae)
print('RMSE to win min model for phase is : %.4f' % armse)
print('MAE to win min model for phase is : %.4f' % amae)

329/329 [==============================] - 2s 6ms/step
Per iteration time: 0.204935
Model Test data RMSEv: 9685.0404
Model Test data MAEv: 9679.2712
Model Test data RMSEa: 2861.8089
Model Test data MAEa: 2615.5814
329/329 [==============================] - 2s 6ms/step
Per iteration time: 0.206695
Model Test data RMSEv: 9685.3346
Model Test data MAEv: 9679.5647
Model Test data RMSEa: 2861.9339
Model Test data MAEa: 2615.7096
329/329 [==============================] - 2s 6ms/step
Per iteration time: 0.209909
Model Test data RMSEv: 9685.3297
Model Test data MAEv: 9679.5598
Model Test data RMSEa: 2861.9323
Model Test data MAEa: 2615.7072
329/329 [==============================] - 2s 6ms/step
Per iteration time: 0.208452
Model Test data RMSEv: 9685.2559
Model Test data MAEv: 9679.4859
Model Test data RMSEa: 2861.9137
Model Test data MAEa: 2615.6839
329/329 [==============================] - 2s 6ms/step
Per iteration time: 0.207111
Model Test data RMSEv: 9685.3331
Model Test data MAEv: 9679.